In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.optimizers import Adam
from keras.layers import Input, Dense, Dropout, Flatten, AveragePooling2D
from keras.models import Model
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
import keras
import tensorflow as tf  # Import TensorFlow

# Load the paths
TRAIN_PATH = r"C:\Users\laksh\Downloads\ROT_FREE_FARM\dataset\train"
TEST_PATH = r"C:\Users\laksh\Downloads\ROT_FREE_FARM\dataset\test"

datagen = ImageDataGenerator(
    rotation_range = 30,
    zoom_range = .3,
    horizontal_flip = True,
    vertical_flip = True,
    validation_split = .3
)

train_ds = datagen.flow_from_directory(
    directory = TRAIN_PATH,
    target_size = (256, 256),
    color_mode = 'rgb',
    class_mode = 'categorical',
    subset = 'training'
)

validation_ds = datagen.flow_from_directory(
    directory = TRAIN_PATH,
    target_size = (256, 256),
    color_mode = 'rgb',
    class_mode = 'categorical',
    subset = 'validation'
)

# Load the class labels
class_labels = list(train_ds.class_indices.keys())

# Load the model without weights
model = load_model("Rotten_fruit_detection_model.h5", compile=False)

# Define the new input shape
new_input_shape = (256, 256, 3)

# Create a new VGG16 model with the modified input shape
new_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=new_input_shape)
new_vgg16.trainable = False  # Set VGG16 to be non-trainable

# Build a new model by combining the new VGG16 and the remaining part of the original model
new_model = keras.models.Sequential([
    new_vgg16,
    model.get_layer('AvgPool2D'),  
    model.get_layer('Flatten'),    
    model.get_layer('Dense1'),  
    model.get_layer('dropout'),    
    model.get_layer('Dense2'),    
    model.get_layer('dropout_1'),  
    model.get_layer('Dense3')      
])

# Compile the new model
new_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Set the background color of the app
app_bg_color = "#ffffff"

# Create the main window
root = tk.Tk()
root.title("ROT FREE FARM🍎🍌🍊")
root.configure(bg=app_bg_color)

# Create frames for home and analyze screens
home_frame = tk.Frame(root, bg=app_bg_color)
analyze_frame = tk.Frame(root, bg=app_bg_color)

# Create canvas for image display
canvas = tk.Canvas(analyze_frame, width=256, height=256, bg="white")
canvas.pack()

# Create a global variable to hold the ImageTk.PhotoImage object
tk_img = None

def upload_image():
    global tk_img
    global image_item
    file_path = filedialog.askopenfilename(title="Select Image", filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        img = Image.open(file_path)
        img.thumbnail((256, 256))
        tk_img = ImageTk.PhotoImage(img)
       
        # Clear previous image
        canvas.delete("all")
       
        # Display the new image on the canvas
        canvas.create_image(128, 128, anchor="center", image=tk_img)
        label_upload.config(text="Image Uploaded")

        # Save image path for later use
        analyze_image.image_path = file_path

# Define the function to switch between screens
def switch_screen():
    if analyze_frame.winfo_ismapped():
        analyze_frame.pack_forget()
        home_frame.pack()
    else:
        home_frame.pack_forget()
        analyze_frame.pack()

# Function to analyze the image
def analyze_image():
    if hasattr(analyze_image, 'image_path'):
        img = Image.open(analyze_image.image_path)
        img = img.resize((256, 256))  # Resize the image to the model's expected input shape
        img_array = np.array(img)  # Convert the image to a NumPy array
        img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
        img_array = preprocess_input(img_array)  # Preprocess the input image

        predictions = new_model.predict(img_array)
        predicted_class_index = np.argmax(predictions)
        predicted_class_label = class_labels[predicted_class_index]

        result_label.config(text=f'Prediction: {predicted_class_label}')
    else:
        result_label.config(text="Please upload an image first")


# Function to switch between home and analyze screens
def logo_click():
    switch_screen()

# Create a welcome message along with the logo in the center of the home screen
app_name_label = tk.Label(home_frame, text="Welcome to ROT FREE FARM", font=("Helvetica", 20, "bold"), bg=app_bg_color)
app_name_label.pack(pady=10)

# Load and display the logo on the home screen
logo_img = Image.open("Green Vintage Agriculture and Farming Logo.png")
logo_img = ImageTk.PhotoImage(logo_img)
logo_label = tk.Label(home_frame, image=logo_img, bg=app_bg_color)
logo_label.pack(pady=20)
logo_label.bind("<Button-1>", lambda e: logo_click())

# Create labels, buttons, and canvas for the analyze screen
app_title_label = tk.Label(analyze_frame, text="ROT FREE FARM🍎🍌🍊", font=("Helvetica", 16, "bold"), bg=app_bg_color)
app_title_label.pack(pady=10)

label_upload = tk.Label(analyze_frame, text="Upload Image:", font=("Helvetica", 16), bg=app_bg_color)
label_upload.pack(pady=10)

btn_upload = tk.Button(analyze_frame, text="Browse", command=upload_image, font=("Helvetica", 14))
btn_upload.pack(pady=10)

btn_analyze = tk.Button(analyze_frame, text="Analyze", command=analyze_image, font=("Helvetica", 14))
btn_analyze.pack(pady=10)

result_label = tk.Label(analyze_frame, text="", font=("Helvetica", 16), bg=app_bg_color)
result_label.pack(pady=10)

# Initialize variables
analyze_image.image_path = ""
analyze_image.predicted_class_label = ""

# Show the home screen initially
home_frame.pack()

# Start the Tkinter event loop
root.mainloop()

Found 7634 images belonging to 6 classes.
Found 3267 images belonging to 6 classes.
